# Lecture Notebook: Big Data and Graph Data

Apache Spark is a big data engine that runs on compute clusters, including on the cloud.  Since not everyone will have access to a compute cluster, this version of the notebook is set up to run Spark locally.


In [0]:
!apt install libkrb5-dev
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install sparkmagic
!pip install pyspark

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  comerr-dev krb5-multidev libcom-err2 libgssrpc4 libkadm5clnt-mit11
  libkadm5srv-mit11 libkdb5-9
Suggested packages:
  doc-base krb5-doc krb5-user
The following NEW packages will be installed:
  comerr-dev krb5-multidev libgssrpc4 libkadm5clnt-mit11 libkadm5srv-mit11
  libkdb5-9 libkrb5-dev
The following packages will be upgraded:
  libcom-err2
1 upgraded, 7 newly installed, 0 to remove and 24 not upgraded.
Need to get 358 kB of archives.
After this operation, 1,992 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libcom-err2 amd64 1.44.1-1ubuntu1.3 [8,848 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libgssrpc4 amd64 1.16-2ub

In [0]:
    import pyspark
    from pyspark.sql import SparkSession
    from pyspark.sql.types import *
    import pyspark.sql.functions as F
    from pyspark.sql import SQLContext

In [0]:
    try:
       if(spark == None):
            spark = SparkSession.builder.appName('Graphs').getOrCreate()
            sqlContext=SQLContext(spark)
    except NameError:
        spark = SparkSession.builder.appName('Graphs').getOrCreate()
        sqlContext=SQLContext(spark)


## Example of Loading Sharded Data

First let's load the data.

In [0]:
import json
import requests

In [0]:
# From https://stackoverflow.com/questions/53575893/download-zip-folder-with-python-3-5-using-urllib
def downloadFile(url):
    local_filename = url.split("/")[len(url.split("/")) -1]
    req = requests.get(url, stream=True)
    with open(local_filename, 'wb') as fl:
        for chunk in req.iter_content(chunk_size=10240): 
            if chunk: 
                fl.write(chunk)
    print ('Wrote %s' %local_filename)

downloadFile('https://github.com/odpi/OpenDS4All/raw/master/assets/data/test_data_10000.zip')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import zipfile

with zipfile.ZipFile('test_data_10000.zip', 'r') as zipObj:
    zipObj.extractall()
    
linked_in = open('test_data_10000.json')
my_list = [json.loads(line) for line in linked_in]

len(my_list)


10000

## Load the list into Spark

Spark needs to know the structure of the data in its dataframes, i.e., their schemas.  Since our JSON structure for LinkedIn is complex, we need to define the schema.

There are some basic types:
  * The table is a `StructType` with a list of fields (each row)
  * Most fields, in our case, are `StringType`.
  * We also have nested dictionary for the name, which is a `MapType` from `StringType` keys to `StringType` values.
  * `skills` is an `ArrayType` since it's a list, and it contains `StringType`s.
  * `also_view` is an array of structs.

See Pyspark documentation on `StructType` and examples such as https://www.programcreek.com/python/example/104715/pyspark.sql.types.StructType.

In [0]:
# Spark requires that we define a schema for the LinkedIn data...
from pyspark.sql.types import StringType, StructField, StructType, ArrayType, MapType
schema = StructType([
        StructField("_id", StringType(), True),
        StructField("name", MapType(StringType(), StringType()), True),
        StructField("locality", StringType(), True),
        StructField("skills", ArrayType(StringType()), True),
        StructField("industry", StringType(), True),
        StructField("summary", StringType(), True),
        StructField("url", StringType(), True),
        StructField("also_view", ArrayType(\
                    StructType([\
                      StructField("url", StringType(), True),\
                      StructField("id", StringType(), True)])\
                    ), True)\
         ])

In [0]:
# Load the remote data as a list of dictionaries
linked_df = sqlContext.createDataFrame(my_list, schema).\
      repartition('_id')

linked_df.show(5)

+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                 _id|                name|           locality|              skills|            industry|             summary|                 url|           also_view|
+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|nvgdapwhxjjzsvycn...|[given_name -> M ...|    Uppsala, Sweden|                  []|    Ämter & Behörden|Ashley Dunning is...|jzvdtmrfybdqgeghl...|[[http://uk.linke...|
|uqerqpzcdggyrnqun...|[given_name -> Ab...|          Singapore|[{value=Motion Co...|Information Techn...|Regional Sales Ma...|cvwkibsajmjkesjvy...|[[http://in.linke...|
|wyzoingfdqnkqrvuq...|[given_name -> Ad...|Houston, Texas Area|[{value=Managemen...|Pengambilan Kakit...|I am a training a...|babrthcyozvewpzkz...|[[http:/

In [0]:
linked_df.select('name').show( truncate = False)

+------------------------------------------------------------------+
|name                                                              |
+------------------------------------------------------------------+
|[given_name -> M Burak, family_name -> Bhandari]                  |
|[given_name -> Abdelhay, family_name -> da Silva dos Santos]      |
|[given_name -> Adnan Maqbool - SAP, family_name -> Masloski]      |
|[given_name -> Anbazhagan (Anbu), family_name -> Aguinaga Azpiazu]|
|[given_name -> Steve, family_name -> Geisler]                     |
|[given_name -> Aysel, family_name -> Singer]                      |
|[given_name -> Abraham, family_name -> Asgeirsson]                |
|[given_name -> Agatha, family_name -> Choung]                     |
|[given_name -> Sean, family_name -> Ekelund]                      |
|[given_name -> Aarushi, family_name -> Åkesson]                   |
|[given_name -> Andrew A., family_name -> Baahmed]                 |
|[given_name -> Agner, family_name

In [0]:
linked_df.rdd.getNumPartitions()

200

In [0]:
linked_df.filter(linked_df.locality == 'United States')[['_id', 'name', 'locality']].show(5)

+--------------------+--------------------+-------------+
|                 _id|                name|     locality|
+--------------------+--------------------+-------------+
|dslvikjojvqncpxag...|[given_name -> Mo...|United States|
|eavnmharasuudkshz...|[given_name -> Je...|United States|
|zmjjbbtwfssfsgxnq...|[given_name -> Po...|United States|
|miagrwbtyjfnrhdvg...|[given_name -> Ag...|United States|
|pfxsthrqjqfzkvlkf...|[given_name -> Ad...|United States|
+--------------------+--------------------+-------------+
only showing top 5 rows



In [0]:
linked_df.select("_id", "locality").show(5)

+--------------------+-------------------+
|                 _id|           locality|
+--------------------+-------------------+
|nvgdapwhxjjzsvycn...|    Uppsala, Sweden|
|uqerqpzcdggyrnqun...|          Singapore|
|wyzoingfdqnkqrvuq...|Houston, Texas Area|
|yitfbojzyhfmhdwdx...| Paris Area, France|
|gliizzsriqmgyfqop...|            Türkiye|
+--------------------+-------------------+
only showing top 5 rows



In [0]:
### Clean out the list from memory
my_list = []

In [0]:
linked_df.createOrReplaceTempView('linked_in')
sqlContext.sql('select * from linked_in').show(5)

+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                 _id|                name|           locality|              skills|            industry|             summary|                 url|           also_view|
+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|nvgdapwhxjjzsvycn...|[given_name -> M ...|    Uppsala, Sweden|                  []|    Ämter & Behörden|Ashley Dunning is...|jzvdtmrfybdqgeghl...|[[http://uk.linke...|
|uqerqpzcdggyrnqun...|[given_name -> Ab...|          Singapore|[{value=Motion Co...|Information Techn...|Regional Sales Ma...|cvwkibsajmjkesjvy...|[[http://in.linke...|
|wyzoingfdqnkqrvuq...|[given_name -> Ad...|Houston, Texas Area|[{value=Managemen...|Pengambilan Kakit...|I am a training a...|babrthcyozvewpzkz...|[[http:/

In [0]:
sqlContext.sql('select _id, name.given_name, name.family_name from linked_in').show(10)

+--------------------+-------------------+-------------------+
|                 _id|         given_name|        family_name|
+--------------------+-------------------+-------------------+
|nvgdapwhxjjzsvycn...|            M Burak|           Bhandari|
|uqerqpzcdggyrnqun...|           Abdelhay|da Silva dos Santos|
|wyzoingfdqnkqrvuq...|Adnan Maqbool - SAP|           Masloski|
|yitfbojzyhfmhdwdx...|  Anbazhagan (Anbu)|   Aguinaga Azpiazu|
|gliizzsriqmgyfqop...|              Steve|            Geisler|
|nilkycgqibqzhcpws...|              Aysel|             Singer|
|cbeqtauvkrpwjdxno...|            Abraham|         Asgeirsson|
|aamexaalrxcxvcvmt...|             Agatha|             Choung|
|ojpxbwcehftbsicle...|               Sean|            Ekelund|
|bpfnuxzybuusoiubr...|            Aarushi|            Åkesson|
+--------------------+-------------------+-------------------+
only showing top 10 rows



In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

acro = udf(lambda x: ''.join([n[0] for n in x.split()]), StringType())

linked_df.select("_id", acro("locality").alias("acronym")).show(5)

+--------------------+-------+
|                 _id|acronym|
+--------------------+-------+
|nvgdapwhxjjzsvycn...|     US|
|uqerqpzcdggyrnqun...|      S|
|wyzoingfdqnkqrvuq...|    HTA|
|yitfbojzyhfmhdwdx...|    PAF|
|gliizzsriqmgyfqop...|      T|
+--------------------+-------+
only showing top 5 rows



In [0]:
# Which industries are most popular?
sqlContext.sql('select count(_id), industry '+\
               'from linked_in '+\
               'group by industry '+\
               'order by count(_id) desc').\
    show(5)

+----------+--------------------+
|count(_id)|            industry|
+----------+--------------------+
|       323|Information Techn...|
|       212|   Computer Software|
|       112|            Internet|
|       102|Marketing and Adv...|
|        72|  Financial Services|
+----------+--------------------+
only showing top 5 rows



## Graphs

For the next set of examples, we will look at graph-structured data.  It turns out our LinkedIn dataset has a list of nodes (by int ID, but associated with the user ID we used in the linked_in table) and a list of edges.

In [0]:
downloadFile('https://github.com/odpi/OpenDS4All/raw/master/assets/data/linkedin.edges.zip')
    
import zipfile

with zipfile.ZipFile('linkedin.edges.zip', 'r') as zipObj:
    zipObj.extractall()



In [ ]:
fname = open('linkedin.edges','r')
edges = []
MAX = 100000

for link in fname:
  edge = link.split(' ')
  edges.append([int(edge[0]), int(edge[1])])
  if len(edges) >= MAX:
    break

In [0]:
from pyspark.sql.types import IntegerType
schema = StructType([
        StructField("from", IntegerType(), True),
        StructField("to", IntegerType(), True)
         ])
# Load the remote data as a list of dictionaries
edges_df = sqlContext.createDataFrame(edges, schema)

edges_df.show(5)

+----+-------+
|from|     to|
+----+-------+
|   0|2152448|
|   0|1656491|
|   0| 399364|
|   0|  18448|
|   0|  72025|
+----+-------+
only showing top 5 rows



In [0]:
edges_df.createOrReplaceTempView('edges')
sqlContext.sql('select from as id, count(to) as degree from edges group by from').show(5)

+---+------+
| id|degree|
+---+------+
|148|   140|
|463|    93|
|471|    88|
|496|    88|
|833|    76|
+---+------+
only showing top 5 rows



## Traversing the Graph

In [0]:
from pyspark.sql.functions import col

# Start with a subset of nodes
start_nodes_df = edges_df[['from']].filter(edges_df['from'] < 1000).\
  select(col('from').alias('id')).drop_duplicates()

start_nodes_df.show(5)

# The neighbors require us to join
# and we'll use Spark DataFrames syntax here
neighbor_nodes_df = start_nodes_df.\
  join(edges_df, start_nodes_df.id == edges_df['from']).\
  select(col('to').alias('id'))

neighbor_nodes_df.show(5)

+---+
| id|
+---+
|148|
|463|
|471|
|496|
|833|
+---+
only showing top 5 rows

+-------+
|     id|
+-------+
|1510404|
|    523|
| 993804|
| 469009|
| 232979|
+-------+
only showing top 5 rows



In [0]:
edges_df[['from']].orderBy('from').drop_duplicates().show()

edges_df.filter(edges_df['from'] == 1).show()

+----+
|from|
+----+
|   0|
|   1|
|   2|
|   3|
|   4|
|   5|
|   6|
|   7|
|   8|
|   9|
|  10|
|  11|
|  12|
|  13|
|  14|
|  15|
|  16|
|  17|
|  18|
|  19|
+----+
only showing top 20 rows

+----+-------+
|from|     to|
+----+-------+
|   1|  77832|
|   1| 542731|
|   1| 317452|
|   1|  27650|
|   1|2662416|
|   1| 104468|
|   1| 176149|
|   1|     25|
|   1|  53282|
|   1| 516132|
|   1|  47142|
|   1| 104488|
|   1| 262186|
|   1|1392685|
|   1| 523471|
|   1| 110639|
|   1| 700465|
|   1|1941562|
|   1| 116809|
|   1|1837130|
+----+-------+
only showing top 20 rows



In [0]:
neighbor_neighbor_nodes_df = neighbor_nodes_df.\
  join(edges_df, neighbor_nodes_df.id == edges_df['from']).\
  select(col('to').alias('id'))

neighbor_neighbor_nodes_df.show(5)

+-------+
|     id|
+-------+
| 445099|
| 435723|
|1666062|
| 390673|
|2328084|
+-------+
only showing top 5 rows



In [0]:
def iterate(df, depth):
  df.createOrReplaceTempView('iter')

  # Base case: direct connection
  result = sqlContext.sql('select from, to, 1 as depth from iter')

  for i in range(1, depth):
    result.createOrReplaceTempView('result')
    result = sqlContext.sql('select r1.from as from, r2.to as to, r1.depth+1 as depth  '\
                            'from result r1 join iter r2 '\
                            'on r1.to=r2.from')
  return result

In [0]:
iterate(edges_df.filter(edges_df['from'] < 1000), 1).orderBy('from','to').show()

+----+----+-----+
|from|  to|depth|
+----+----+-----+
|   0|  38|    1|
|   0| 101|    1|
|   0| 121|    1|
|   0| 161|    1|
|   0| 337|    1|
|   0| 487|    1|
|   0| 504|    1|
|   0| 802|    1|
|   0|1379|    1|
|   0|1583|    1|
|   0|1961|    1|
|   0|1982|    1|
|   0|1996|    1|
|   0|2250|    1|
|   0|2409|    1|
|   0|2692|    1|
|   0|3179|    1|
|   0|3250|    1|
|   0|3787|    1|
|   0|4213|    1|
+----+----+-----+
only showing top 20 rows



In [0]:
iterate(edges_df.filter(edges_df['from'] < 1000), 2).orderBy('from','to').show()

+----+---+-----+
|from| to|depth|
+----+---+-----+
|   0| 59|    2|
|   0| 66|    2|
|   0|101|    2|
|   0|121|    2|
|   0|121|    2|
|   0|161|    2|
|   0|236|    2|
|   0|236|    2|
|   0|236|    2|
|   0|337|    2|
|   0|337|    2|
|   0|337|    2|
|   0|487|    2|
|   0|487|    2|
|   0|487|    2|
|   0|487|    2|
|   0|504|    2|
|   0|504|    2|
|   0|504|    2|
|   0|504|    2|
+----+---+-----+
only showing top 20 rows



In [0]:
iterate(edges_df.filter(edges_df['from'] < 1000), 3).orderBy('from','to').show()

+----+---+-----+
|from| to|depth|
+----+---+-----+
|   0|101|    3|
|   0|101|    3|
|   0|121|    3|
|   0|121|    3|
|   0|121|    3|
|   0|236|    3|
|   0|236|    3|
|   0|236|    3|
|   0|337|    3|
|   0|337|    3|
|   0|337|    3|
|   0|337|    3|
|   0|337|    3|
|   0|337|    3|
|   0|337|    3|
|   0|337|    3|
|   0|487|    3|
|   0|487|    3|
|   0|487|    3|
|   0|487|    3|
+----+---+-----+
only showing top 20 rows



In [0]:
# Clear list of edges from Python memory
# to free up space
edges = []


### Now let's get the list of node IDs
downloadFile('https://raw.githubusercontent.com/odpi/OpenDS4All/master/assets/data/linkedin.nodes.zip')

zip_file_object = zipfile.ZipFile('linkedin.nodes.zip', 'r')
fname = zip_file_object.open('linkedin.nodes')

nodes = []
MAX = 100000

for node in fname:
  node_tuple = node.decode('utf-8').split()
  nodes.append([int(node_tuple[0]), str(node_tuple[1])])
  if len(nodes) >= MAX:
    break


## Joins in Spark, Beyond Graph Traversals

What if we want to connect our edges to the people from our previous crawl?  Sadly the edges use int node IDs that don't correspond to the people dataframe.  But in fact the node data includes this information, so let's load and exploit that.

Let's load the information about nodes, and their correspondence to the user ID.

In [0]:
schema = StructType([
        StructField("nid", IntegerType(), True),
        StructField("user", StringType(), True)
         ])
# Load the remote data as a list of dictionaries
nodes_df = sqlContext.createDataFrame(nodes, schema)

nodes_df.show(5)

+---+--------------------+
|nid|                user|
+---+--------------------+
|  0|pub-sandra-arana-...|
|  1|     in-sehrishhafiz|
|  2|pub-heba-bayoumi-...|
|  3|pub-aysha-binbrai...|
|  4|      in-rubabadowla|
+---+--------------------+
only showing top 5 rows



## Finding Friends, by ID

In [0]:
nodes_df.createOrReplaceTempView('nodes')
edges_df.createOrReplaceTempView('edges')

friends_df = \
sqlContext.sql('select n1.user, n2.user as friend ' +\
               'from (nodes n1 join edges e on n1.nid = e.from) join nodes n2 on e.to = n2.nid')

friends_df.show(5)


+--------------------+--------------------+
|                user|              friend|
+--------------------+--------------------+
|pub-sara-muÃ±oz-r...|pub-rebeca-arroyo...|
|pub-gloria-villal...|pub-rebeca-arroyo...|
|in-dianacolliabugeda|pub-rebeca-arroyo...|
|        in-maitepena|pub-rebeca-arroyo...|
|         in-alvarovl|pub-begoÃ±a-landa...|
+--------------------+--------------------+
only showing top 5 rows



## Connecting Friends to Names

In [0]:
friends_df.createOrReplaceTempView('friends')

sqlContext.sql('select u1.name.given_name as user, u2.name.given_name as friend '+\
               'from (linked_in u1 join friends on u1._id = user) join linked_in u2 on u2._id = friend').show(5)

+----+------+
|user|friend|
+----+------+
|Adam|  Abiy|
+----+------+

